In [1]:
from torch import nn, optim
from torch.utils.data import DataLoader
import torch
import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn.functional as F
from scipy.stats import pearsonr
from train import train_model, test_model
from model_BiLSTM_v1 import BiLSTM
from torch.utils.tensorboard import SummaryWriter

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [3]:
new_deepmass_df = pd.read_csv("./data/renewal_deepmass.tsv", sep="\t", low_memory=False)
new_hela1_df = pd.read_csv("./data/renewal_hela1.tsv", sep="\t", low_memory=False)
new_hela2_df = pd.read_csv("./data/renewal_hela2.tsv", sep="\t", low_memory=False)

max_seq_len = 35
max_intens_len = 70

In [4]:
# model setting
input_size = 1
hidden_size = 128
num_layers = 2
output_size = max_intens_len  # 출력 크기는 인텐시티 길이


In [5]:
class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()
        self.cosine_similarity = nn.CosineSimilarity(dim=1, eps=1e-6)

    def forward(self, y_pred, y_true):
        return 1 - self.cosine_similarity(y_pred, y_true).mean()

# 손실 함수 및 옵티마이저
criterion = CosineSimilarityLoss()


### Version2_trial1: DeepMass(20%) pre-training and pine tuning with 20% of Hela1
  - pre-training=70:30
  - fine-tuning=60:20:20

In [11]:
model = BiLSTM(input_size, hidden_size, num_layers, output_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)

tb_writer = SummaryWriter("./tensorboard_logs/version2/trial1-pre_training")

# pre-training dataset
sampled_df = new_deepmass_df.sample(frac=0.6, random_state=44)
pre_train_df, pre_val_df = train_test_split(sampled_df, test_size=0.2, random_state=44)
print(f"total df: {len(sampled_df)}, train df: {len(pre_train_df)}, val df: {len(pre_val_df)}")
pre_train_dataset = preprocessing.retrieve_dataset(pre_train_df, max_seq_len, max_intens_len, normalize_intens=True)
pre_val_dataset = preprocessing.retrieve_dataset(pre_val_df, max_seq_len, max_intens_len, normalize_intens=True)
pre_train_dataloader = DataLoader(pre_train_dataset, batch_size=2048, shuffle=True)
pre_val_dataloader = DataLoader(pre_val_dataset, batch_size=128, shuffle=True)

# fine tuning dataset
tuning_train_df, tuning_val_test_df = train_test_split(new_hela1_df, test_size=0.4, random_state=44)
tuning_val_df, tuning_test_df = train_test_split(tuning_val_test_df, test_size=0.5, random_state=44)
tuning_train_dataset = preprocessing.retrieve_dataset(tuning_train_df, max_seq_len, max_intens_len, normalize_intens=True)
tuning_val_dataset = preprocessing.retrieve_dataset(tuning_val_df, max_seq_len, max_intens_len, normalize_intens=True)
tuning_test_dataset = preprocessing.retrieve_dataset(tuning_test_df, max_seq_len, max_intens_len, normalize_intens=True)
tuning_train_dataloader = DataLoader(tuning_train_dataset, batch_size=2048, shuffle=True)
tuning_val_dataloader = DataLoader(tuning_val_dataset, batch_size=128, shuffle=True)
tuning_test_dataloader = DataLoader(tuning_test_dataset, batch_size=128, shuffle=True)

# other test dataset
hela2_dataset = preprocessing.retrieve_dataset(new_hela2_df, max_seq_len, max_intens_len, normalize_intens=True)
hela2_dataloader = DataLoader(hela2_dataset, batch_size=128, shuffle=True)

# pre-training
epochs = 300
pre_trained_model, pre_train_res = train_model(model, criterion, optimizer, epochs, pre_train_dataloader, pre_val_dataloader, device, tb_writer, 40)
tb_writer.close()
# freeze
for name, param in pre_trained_model.named_parameters():
    if "fc" not in name:
        param.requires_grad = False
torch.save(pre_trained_model.state_dict(), './pre_trained_model_parameters_60_80.pth')

total df: 339276, train df: 271420, val df: 67856
Epoch [1/300], Train Loss: 0.2681, Validation Loss: 0.2358
Train Cosine Similarity: 0.7318, Train PCC: 0.6858
Validation Cosine Similarity: 0.7642, Validation PCC: 0.7213
Epoch [2/300], Train Loss: 0.2120, Validation Loss: 0.2088
Train Cosine Similarity: 0.7880, Train PCC: 0.7482
Validation Cosine Similarity: 0.7911, Validation PCC: 0.7518
Epoch [3/300], Train Loss: 0.2070, Validation Loss: 0.2066
Train Cosine Similarity: 0.7930, Train PCC: 0.7540
Validation Cosine Similarity: 0.7935, Validation PCC: 0.7545
Epoch [4/300], Train Loss: 0.2053, Validation Loss: 0.2041
Train Cosine Similarity: 0.7947, Train PCC: 0.7559
Validation Cosine Similarity: 0.7958, Validation PCC: 0.7571
Epoch [5/300], Train Loss: 0.2021, Validation Loss: 0.1993
Train Cosine Similarity: 0.7979, Train PCC: 0.7594
Validation Cosine Similarity: 0.8008, Validation PCC: 0.7625
Epoch [6/300], Train Loss: 0.1978, Validation Loss: 0.1943
Train Cosine Similarity: 0.8022, Tra

In [13]:
tb_writer = SummaryWriter("./tensorboard_logs/version2/trial1-fine_tuning")
epochs = 300
optimizer = optim.Adam(pre_trained_model.parameters(), lr=0.0005)
tuning_model, tuning_train_res = train_model(pre_trained_model, criterion, optimizer, epochs, tuning_train_dataloader, tuning_val_dataloader, device, tb_writer, 40)

test_res = test_model(tuning_model, criterion, tuning_test_dataloader, device, tb_writer, "hela 20% test")
hela2_res = test_model(tuning_model, criterion, hela2_dataloader, device, tb_writer, "hela2 test")
tb_writer.close()

Epoch [1/300], Train Loss: 0.2583, Validation Loss: 0.3634
Train Cosine Similarity: 0.7417, Train PCC: 0.7146
Validation Cosine Similarity: 0.6370, Validation PCC: 0.5977
Epoch [2/300], Train Loss: 0.2563, Validation Loss: 0.3603
Train Cosine Similarity: 0.7437, Train PCC: 0.7168
Validation Cosine Similarity: 0.6393, Validation PCC: 0.6002
Epoch [3/300], Train Loss: 0.2543, Validation Loss: 0.3572
Train Cosine Similarity: 0.7457, Train PCC: 0.7190
Validation Cosine Similarity: 0.6416, Validation PCC: 0.6026
Epoch [4/300], Train Loss: 0.2523, Validation Loss: 0.3563
Train Cosine Similarity: 0.7476, Train PCC: 0.7211
Validation Cosine Similarity: 0.6438, Validation PCC: 0.6050
Epoch [5/300], Train Loss: 0.2505, Validation Loss: 0.3543
Train Cosine Similarity: 0.7495, Train PCC: 0.7232
Validation Cosine Similarity: 0.6459, Validation PCC: 0.6073
Epoch [6/300], Train Loss: 0.2487, Validation Loss: 0.3511
Train Cosine Similarity: 0.7514, Train PCC: 0.7253
Validation Cosine Similarity: 0.648

In [17]:
pre_trained_model = BiLSTM(input_size, hidden_size, num_layers, output_size).to(device)
pre_trained_model.load_state_dict(torch.load("./pre_trained_model_parameters_60_80.pth", map_location=torch.device(device)))

tb_writer = SummaryWriter("./tensorboard_logs/version2/trial1-fine_tuning_#2")
epochs = 300
optimizer = optim.Adam(pre_trained_model.parameters(), lr=0.0001)
tuning_model, tuning_train_res = train_model(pre_trained_model, criterion, optimizer, epochs, tuning_train_dataloader, tuning_val_dataloader, device, tb_writer, 40)

test_res = test_model(tuning_model, criterion, tuning_test_dataloader, device, tb_writer, "hela 20% test")
hela2_res = test_model(tuning_model, criterion, hela2_dataloader, device, tb_writer, "hela2 test")
tb_writer.close()

Epoch [1/300], Train Loss: 0.2764, Validation Loss: 0.3603
Train Cosine Similarity: 0.7234, Train PCC: 0.6949
Validation Cosine Similarity: 0.6401, Validation PCC: 0.6016
Epoch [2/300], Train Loss: 0.2313, Validation Loss: 0.3164
Train Cosine Similarity: 0.7685, Train PCC: 0.7445
Validation Cosine Similarity: 0.6820, Validation PCC: 0.6473
Epoch [3/300], Train Loss: 0.1913, Validation Loss: 0.2792
Train Cosine Similarity: 0.8085, Train PCC: 0.7887
Validation Cosine Similarity: 0.7200, Validation PCC: 0.6891
Epoch [4/300], Train Loss: 0.1583, Validation Loss: 0.2459
Train Cosine Similarity: 0.8415, Train PCC: 0.8253
Validation Cosine Similarity: 0.7516, Validation PCC: 0.7239
Epoch [5/300], Train Loss: 0.1348, Validation Loss: 0.2255
Train Cosine Similarity: 0.8650, Train PCC: 0.8513
Validation Cosine Similarity: 0.7745, Validation PCC: 0.7492
Epoch [6/300], Train Loss: 0.1202, Validation Loss: 0.2101
Train Cosine Similarity: 0.8797, Train PCC: 0.8674
Validation Cosine Similarity: 0.790